In [14]:
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os
import pandas as pd

In [6]:
csv_wmi = pd.read_csv("normal01/normal_wmi.csv", index_col = 0)
csv_wmi = csv_wmi.drop(["Name"], axis = 1)

scaler = MinMaxScaler()
scaled_wmi = scaler.fit_transform(csv_wmi.to_numpy())

<h3>isolaion forest</h3>

In [7]:
isol10 = IsolationForest(contamination = 0.005, max_samples=100, random_state = 0)
isol10.fit(scaled_wmi)

IsolationForest(contamination=0.005, max_samples=100, random_state=0)

In [8]:
total_wmi = pd.read_csv("wmi_total.csv", index_col = 0)
csv_wmi = total_wmi.drop(["malicious"], axis = 1)

y = total_wmi['malicious'].values
y = pd.DataFrame(y, columns = ["label"])

scaler = MinMaxScaler()
test_wmi = scaler.fit_transform(csv_wmi.to_numpy())

In [11]:
y_pred = isol10.predict(test_wmi)

<h3>one-class svm</h3>

In [12]:
svm10 = svm.OneClassSVM(nu = 0.002, kernel = "rbf", gamma = 'scale')

svm10.fit(scaled_wmi)

OneClassSVM(nu=0.002)

In [13]:
y_pred_svm = svm10.predict(test_wmi)

<h3>PCA</h3>

In [15]:
def read_dir():
    path = "infected system/"
    listdir = os.listdir(path)
    
    return listdir

In [16]:
def anomaly_pca():
    listdir = read_dir()
    result = []
    total_df = pd.DataFrame(columns = ['mse', 'label'])
    for d in listdir:
        print("============================================================")
        print(d)
        df = pd.read_csv("infected system/" + d + "/wmi.csv", index_col = 0)
        x = df.drop(["Name", "f30"], axis = 1).values
        y = df['f30'].values
        
        x = StandardScaler().fit_transform(x)
        
        recovered_x = pca(x)
        nr = cal_mse(pd.DataFrame(x), pd.DataFrame(recovered_x), y, d)
        sorted_df = cal_mse_3333(pd.DataFrame(x), pd.DataFrame(recovered_x), y, d)
        
        total_df = pd.concat([total_df, sorted_df], axis = 0)
        
        result.append(nr)
        
    #return result
    return total_df

In [17]:
def pca(x):
    pca = PCA(n_components=5)
    printcipalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data=printcipalComponents)
    
    recovered_x = pca.inverse_transform(printcipalComponents)
    recovered_x_df = pd.DataFrame(recovered_x)
    
    return recovered_x

In [18]:
def cal_mse(df1, df2, y, d):
    mse = []

    for i in range(0, len(df1)):
        mse.append(((df1.loc[i] - df2.loc[i]) ** 2).sum() / len(df1.loc[i]))
        
    mseDF = pd.DataFrame(mse, columns = ["mse"])
    labelDF = pd.DataFrame(y, columns = ["label"])
    mse_label = pd.concat([mseDF, labelDF],axis=1)
    sorted_df = mse_label.sort_values(by = 'mse', ascending = False)
    sorted_df = sorted_df.reset_index(drop = True)

    t10 = 0
    t20 = 0
    t30 = 0
    tmal = 0
    
    for i in range(0, len(sorted_df)):
        if sorted_df.loc[i]["label"] == 1:
            print(str(i) + "/" + str(len(df1)))
            tmal = tmal + 1
            if i <= len(sorted_df)/20:
                t10 = t10 + 1
            if i <= len(sorted_df)/10:
                t20 = t20 + 1
            if i <= len(sorted_df)/7:
                t30 = t30 + 1
    
    prc10 = t10 / (len(sorted_df)/20)
    prc20 = t20 / (len(sorted_df)/10)
    prc30 = t30 / (len(sorted_df)/7)
    
    if tmal != 0:
        rcl10 = t10 / tmal
        rcl20 = t20 / tmal
        rcl30 = t30 / tmal
        
    else:
        rcl10 = 0
        rcl20 = 0
        rcl30 = 0
    
    if (prc10 + rcl10) != 0:
        fs10 = 2 * (prc10 * rcl10) / (prc10 + rcl10)
    else:
        fs10 = 0
    if (prc20 + rcl20) != 0:
        fs20 = 2 * (prc20 * rcl20) / (prc20 + rcl20)
    else:
        fs20 = 0
    if (prc30 + rcl30) != 0:
        fs30 = 2 * (prc30 * rcl30) / (prc30 + rcl30)
    else:
        fs30 = 0
    
    return [d, prc10, rcl10, fs10, prc20, rcl20, fs20, prc30, rcl30, fs30]

In [20]:
def cal_mse_3333(df1, df2, y, d):
    mse = []

    for i in range(0, len(df1)):
        mse.append(((df1.loc[i] - df2.loc[i]) ** 2).sum() / len(df1.loc[i]))
        
    mseDF = pd.DataFrame(mse, columns = ["mse"])
    labelDF = pd.DataFrame(y, columns = ["label"])
    mse_label = pd.concat([mseDF, labelDF],axis=1)
    sorted_df = mse_label.sort_values(by = 'mse', ascending = False)
    sorted_df = sorted_df.reset_index(drop = True)
    
    return sorted_df

In [21]:
result = anomaly_pca()

2a198c0616908013a49e856678b5143c
26/153
43/153
4684f096da1f045a4e5c4ee7ca945259
23/147
4996efb901d5c379c575bb7908651071
4aa53791373b8d09b5773f3ea086b803
24/152
5771a7f5aaa45c2a12fa7d1d043ae222
6b8d9642391687d948129206b805b980
46/154
6e2da504218d67df1a4d8db18e9be706
40/151
6ea487c55f789e2d2c0b3c20c8674214
27/159
6eedee0bc42edb6922e6e78f749bab88
39/152
65/152
76/152
78/152
82/152
89/152
92/152
6f194c2bf64aa91f8b25768aaedf954e
6f3c7fa16f0e61f01c9ca873ce257981
7/146
6fd75941954dda0a8d6bd5180b233a37
34/162
8102acd2e76cdc88d8c893ed30e56f46
8821a5f07134fbaacf35ba62ba50aaed
20/155
9629bdb89f6d9fa96162f5d4a0be3e38
2/153
